<a href="https://colab.research.google.com/github/danielpaulMBRDI/danielpaulMBRDI/blob/main/5_Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoEncoders

Autoencoders automatically consists of two structures: the encoder and the decoder. The encoder network downsamples the data into lower dimensions and the decoder network reconstructs the original data from the lower dimension representation. The lower dimension representation is usually called latent space representation. 

![alt text](https://cdn-images-1.medium.com/max/1000/1*8ixTe1VHLsmKB3AquWdxpQ.png)

[Source](https://medium.com/@birla.deepak26/autoencoders-76bb49ae6a8f)

In [ ]:
!pip install --upgrade --no-cache-dir gdown

## Import the libraries

In [ ]:
import torch
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import plotnine
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.autograd import Variable

In [ ]:
what_were_covering = {1: "data (prepare and load)",
    2: "build model",
    3: "fitting the model to data (training)",
    4: "making predictions and evaluating a model (inference)",
    5: "saving and loading a model",
    6: "putting it all together"
}

## 1. Data Preparation

In [ ]:
# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# load the training and test datasets
train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False,
                                  download=True, transform=transform)

In [ ]:
print(len(train_data), 'train samples')
print(len(test_data), 'test samples')

### 1.1 Train and Test Loader

In [ ]:
# Create training and test dataloaders

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)

### 1.2 Visualize Data

In [ ]:
%matplotlib inline
    
# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = next(dataiter)
images = images.numpy()

# get one image from the batch
img = np.squeeze(images[0])

fig = plt.figure(figsize = (5,5)) 
ax = fig.add_subplot(111)
ax.imshow(img, cmap='gray')

## 2. Model Building

# Linear Autoencoder

We'll train an autoencoder with these images by flattening them into 784 length vectors. The images from this dataset are already normalized such that the values are between 0 and 1. Let's start by building a simple autoencoder. The encoder and decoder should be made of one linear layer. The units that connect the encoder and decoder will be the compressed representation.

Since the images are normalized between 0 and 1, we need to use a sigmoid activation on the output layer to get values that match this input value range.

In [ ]:
# define the NN architecture
class Autoencoder(nn.Module):
    def __init__(self, encoding_dim):
        super(Autoencoder, self).__init__()
        ## encoder ##
        self.encoder = nn.Linear(784, encoding_dim)
        ## decoder ##
        self.decoder = nn.Linear(encoding_dim, 784)

    def forward(self, x):
        # define feedforward behavior 
        # and scale the *output* layer with a sigmoid activation function
        
        # pass x into encoder
        out = F.relu(self.encoder(x))
        # pass out into decoder
        out = torch.sigmoid(self.decoder(out))
        
        return out


### 2.1 Check if GPU is available

In [ ]:
torch.cuda.is_available()

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

### 2.2 Initialize Model

In [ ]:
# initialize the NN
encoding_dim = 64
model = Autoencoder(encoding_dim)
model = model.to(device)
print(model)

## 3. Train model

### 3.1 Define a Loss function and optimizer

In [ ]:
# specify loss function
criterion = nn.MSELoss()

# specify loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### 3.2 Train the network

In [ ]:
# number of epochs to train the model
n_epochs = 20

for epoch in range(1, n_epochs+1):
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    for data in train_loader:
        # _ stands in for labels, here
        images, _ = data
        # flatten images
        images = images.view(images.size(0), -1)
        images = images.to(device)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        outputs = model(images)
        # calculate the loss
        loss = criterion(outputs, images)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*images.size(0)
            
    # print avg training statistics 
    train_loss = train_loss/len(train_loader)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch, 
        train_loss
        ))

## 4. Model Evaluation

### 4.1 Visualize the Encoded Inputs

In [ ]:
dataiter = iter(test_loader)
images, labels = next(dataiter)

images_flatten = images.view(images.size(0), -1)
# get sample outputs
images_flatten = images_flatten.to(device)
output = model.encoder(images_flatten)

# output is resized into a batch of images
output = output.view(batch_size, 1, 8, 8)
# use detach when it's an output that requires_grad
output = output.detach().cpu().detach().numpy()

# plot the first ten input images and then reconstructed images
fig, axes = plt.subplots(nrows=1, ncols=10, sharex=True, sharey=True, figsize=(25,4))

for img, ax in zip(images, axes):
    ax.imshow(np.squeeze(img), cmap='gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

fig, axes = plt.subplots(nrows=1, ncols=10, sharex=True, sharey=True, figsize=(25,4))
for img, ax in zip(output, axes):
    ax.imshow(np.squeeze(img), cmap='gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

### 4.2 Visualize Reconstructed Output

In [ ]:
# obtain one batch of test images
dataiter = iter(test_loader)
images, labels = next(dataiter)

images_flatten = images.view(images.size(0), -1)
# get sample outputs
images_flatten = images_flatten.to(device)
output = model(images_flatten)
# prep images for display
images = images.numpy()

# output is resized into a batch of images
output = output.view(batch_size, 1, 28, 28)
# use detach when it's an output that requires_grad
output = output.detach().cpu().detach().numpy()

# plot the first ten input images and then reconstructed images
fig, axes = plt.subplots(nrows=2, ncols=10, sharex=True, sharey=True, figsize=(25,4))

# input images on top row, reconstructions on bottom
for images, row in zip([images, output], axes):
    for img, ax in zip(images, row):
        ax.imshow(np.squeeze(img), cmap='gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

### 4.3 Visualize Distribution of Encodings in 2D Space

#### 4.3.1 Create a t-SNE Plot

In [ ]:
tsne = TSNE(n_components=2)
test_encoded_df = pd.DataFrame(columns = ['Encoding_1', 'Encoding_2', 'Target'])
outputs = []
labels = []

for data in test_loader:
        # _ stands in for labels, here
        images, label = data
        images = images.view(images.size(0), -1)
        images = images.to(device)
        output = model.encoder(images)
        for i in range(0, batch_size):
            # output is resized into a batch of images
            output_n = output[i]
            lab = label[i].item()
            
            outputs.append(output_n.detach().cpu().detach().numpy())
            labels.append(lab)

tsne_op = tsne.fit_transform(outputs)



In [ ]:
for i in range(0, len(labels)):
  test_encoded_df = test_encoded_df.append({'Encoding_1' :tsne_op[i][0] , 'Encoding_2' :tsne_op[i][1], 'Target' : labels[i]},
ignore_index = True)

In [ ]:
cluster_plot_mnist_linear = (
    plotnine.ggplot(data=test_encoded_df)
    + plotnine.geom_point(
        mapping=plotnine.aes(x="Encoding_1", y="Encoding_2", fill="factor(Target)"),
        size=2,
        color="black",
    )
    + plotnine.xlab(xlab="Encoding dimension 1")
    + plotnine.ylab(ylab="Encoding dimension 2")
    + plotnine.ggtitle(title="MNIST Linear autoencoder with 2-dimensional encoding")
    + plotnine.theme_matplotlib()
)


#### 4.3.2 Visualize t-SNE Plot

In [ ]:
cluster_plot_mnist_linear

# Linear Autoencoder with CIFAR 10

## 1. Download Weights of Pretrained model

In [ ]:
!gdown --id 1zEv6kxVzwUvPp0amC3LLlDlsjupvmE1A

## 2. Data Preparation

In [ ]:
# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# load the training and test datasets
train_data = datasets.CIFAR10(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.CIFAR10(root='data', train=False,
                                  download=True, transform=transform)

### 2.1 Train Test split

In [ ]:
# Create training and test dataloaders

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)

### 2.2 Visualize Images

In [ ]:
def imshow(img):
    # img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    img = np.transpose(npimg, (1, 2, 0))
    plt.imshow(img)
    plt.show()


In [ ]:
dataiter = iter(test_loader)
images, labels = next(dataiter)

# print images
imshow(torchvision.utils.make_grid(images))

## 3. Model Building

In [ ]:
# define the NN architecture
class Autoencoder(nn.Module):
    def __init__(self, encoding_dim):
        super(Autoencoder, self).__init__()
        ## encoder ##
        self.encoder = nn.Linear(3072, encoding_dim)
        ## decoder ##
        self.decoder = nn.Linear(encoding_dim, 3072)

    def forward(self, x):
        # define feedforward behavior 
        # and scale the *output* layer with a sigmoid activation function
        
        # pass x into encoder
        out = F.relu(self.encoder(x))
        # pass out into decoder
        out = torch.sigmoid(self.decoder(out))
        
        return out


### 3.1 Check if GPU available

In [ ]:
torch.cuda.is_available()

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

### 3.2 Initialize Model

In [ ]:
# initialize the NN 
# Define Bottleneck
encoding_dim = 0
model = Autoencoder(encoding_dim)
model = model.to(device)
print(model)

### 3.3 Load weights

In [ ]:
# Specify a path
PATH = "cifar_linear.pt"

# Load
model = torch.load(PATH)
model.eval()

## 4. Model Evaluation

### 4.1 Visualize Encodings

In [ ]:
dataiter = iter(test_loader)
images, labels = next(dataiter)

images_flatten = images.view(images.size(0), -1)
# get sample outputs
images_flatten = images_flatten.to(device)
output = model.encoder(images_flatten)

# output is resized into a batch of images
output = output.view(batch_size, 1, 16, 16)
# use detach when it's an output that requires_grad
output = output.detach().cpu().detach().numpy()

# plot the first ten input images and then reconstructed images
fig, axes = plt.subplots(nrows=1, ncols=10, sharex=True, sharey=True, figsize=(25,4))


for img, ax in zip(images, axes):
    npimg = img.numpy()
    # print(npimg.shape)
    img = np.transpose(npimg, (1, 2, 0))
    ax.imshow(img)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

# fig, axes = plt.subplots(nrows=1, ncols=10, sharex=True, sharey=True, figsize=(25,4))
# for img, ax in zip(output, axes):
#     ax.imshow(np.squeeze(img))
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)

### 4.2 Visualize Reconstructed Images

In [ ]:
# obtain one batch of test images
dataiter = iter(test_loader)
images, labels = next(dataiter)

images_flatten = images.view(images.size(0), -1)
# get sample outputs
images_flatten = images_flatten.to(device)
output = model(images_flatten)
# prep images for display
images = images.numpy()

# output is resized into a batch of images
output = output.view(batch_size, 3, 32, 32)
# use detach when it's an output that requires_grad
output = output.detach().cpu().detach().numpy()

# plot the first ten input images and then reconstructed images
fig, axes = plt.subplots(nrows=2, ncols=10, sharex=True, sharey=True, figsize=(25,4))

# input images on top row, reconstructions on bottom
for images, row in zip([images, output], axes):
    for img, ax in zip(images, row):
        # print(npimg.shape)
        img = np.transpose(img, (1, 2, 0))
        ax.imshow(img)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

### 4.3 Visualize t-SNE plots for the embeddings

In [ ]:
tsne = TSNE(n_components=2)
test_encoded_df = pd.DataFrame(columns = ['Encoding_1', 'Encoding_2', 'Target'])
outputs = []
labels = []

for data in test_loader:
        # _ stands in for labels, here
        images, label = data
        images = images.view(images.size(0), -1)
        images = images.to(device)
        output = model.encoder(images)
        for i in range(0, batch_size):
            # output is resized into a batch of images
            output_n = output[i]
            lab = label[i].item()
            
            outputs.append(output_n.detach().cpu().detach().numpy())
            labels.append(lab)

tsne_op = tsne.fit_transform(outputs)

In [ ]:
for i in range(0, len(labels)):
  test_encoded_df = test_encoded_df.append({'Encoding_1' :tsne_op[i][0] , 'Encoding_2' :tsne_op[i][1], 'Target' : labels[i]},
ignore_index = True)

In [ ]:
cluster_plot_cifar_linear = (
    plotnine.ggplot(data=test_encoded_df)
    + plotnine.geom_point(
        mapping=plotnine.aes(x="Encoding_1", y="Encoding_2", fill="factor(Target)"),
        size=2,
        color="black",
    )
    + plotnine.xlab(xlab="Encoding dimension 1")
    + plotnine.ylab(ylab="Encoding dimension 2")
    + plotnine.ggtitle(title="CIFAR Linear autoencoder with 2-dimensional encoding")
    + plotnine.theme_matplotlib()
)


In [ ]:
cluster_plot_cifar_linear

## Convolutional autoencoder

We may also ask ourselves: can autoencoders be used with Convolutions instead of Fully-connected layers ?

The answer is yes and the principle is the same, but using images (3D vectors) instead of flattened 1D vectors. The input image is downsampled to give a latent representation of smaller dimensions and force the autoencoder to learn a compressed version of the images.

## 1. Data Preparation

In [ ]:
# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# load the training and test datasets
train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False,
                                  download=True, transform=transform)

### 1.1 Train Test Loaders

In [ ]:
# Create training and test dataloaders

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)

## 2. Model Building

In [ ]:
class Autoencoder(nn.Module):

    def __init__(self):
        super(Autoencoder,self).__init__()
        
        self.encoder = nn.Sequential(
            # conv layer (depth from 1 --> 16), 3x3 kernels
            nn.Conv2d(1, 16, 3, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            # conv layer (depth from 16 --> 4), 3x3 kernels
            nn.Conv2d(16, 4, 3, padding=1),
            nn.ReLU(True),
            # pooling layer to reduce x-y dims by two; kernel and stride of 2
            nn.MaxPool2d(2, 2))

        self.decoder = nn.Sequential(  
            ## a kernel of 2 and a stride of 2 will increase the spatial dims by 2           
            nn.ConvTranspose2d(4, 16, 2, stride=2),
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 1, 2, stride=2),
            nn.Sigmoid())

    def forward(self,x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

### 2.1 Check if GPU available

In [ ]:
torch.cuda.is_available()

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

### 2.2 Initialize Model

In [ ]:
model = Autoencoder()
model = model.to(device)
print(model)

## 3. Training a Model

### 3.1 Define a Loss function and optimizer

In [ ]:
# specify loss function
criterion = nn.MSELoss()

# specify loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### 3.2 Training a Network

In [ ]:
n_epochs = 20
for epoch in range(n_epochs+1):
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    for data in train_loader:
        # _ stands in for labels, here
        # no need to flatten images
        images, _ = data
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        outputs = model(images.to(device))
        # calculate the loss
        loss = criterion(outputs, images.to(device))
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*images.size(0)
            
    # print avg training statistics 
    train_loss = train_loss/len(train_loader)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch, 
        train_loss
        ))
    

## 4. Model Evaluation

### 4.1 Visualize the Reconstructed Inputs

In [ ]:
# obtain one batch of test images
dataiter = iter(test_loader)
images, labels = next(dataiter)

# get sample outputs
output = model(images.to(device))
# prep images for display
images = images.numpy()

# output is resized into a batch of iages
output = output.view(batch_size, 1, 28, 28)
# use detach when it's an output that requires_grad
output = output.cpu().detach().numpy()

# plot the first ten input images and then reconstructed images
fig, axes = plt.subplots(nrows=2, ncols=10, sharex=True, sharey=True, figsize=(25,4))

# input images on top row, reconstructions on bottom
for images, row in zip([images, output], axes):
    for img, ax in zip(images, row):
        ax.imshow(np.squeeze(img), cmap='gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

### 4.2 Visualize the embeddings using t-SNE plots

In [ ]:
tsne = TSNE(n_components=2)
test_encoded_df = pd.DataFrame(columns = ['Encoding_1', 'Encoding_2', 'Target'])
outputs = []
labels = []

for data in test_loader:
        # _ stands in for labels, here
        images, label = data
        # get sample outputs
        output = model(images.to(device))
        # Make 1D
        output = output.view(images.size(0), -1)
        for i in range(0, batch_size):
            # output is resized into a batch of images
            output_n = output[i]
            lab = label[i].item()
            
            outputs.append(output_n.cpu().detach().numpy())
            labels.append(lab)

tsne_op = tsne.fit_transform(outputs)



In [ ]:
for i in range(0, len(labels)):
  test_encoded_df = test_encoded_df.append({'Encoding_1' :tsne_op[i][0] , 'Encoding_2' :tsne_op[i][1], 'Target' : labels[i]},
ignore_index = True)

In [ ]:
cluster_plot_mnist_conv = (
    plotnine.ggplot(data=test_encoded_df)
    + plotnine.geom_point(
        mapping=plotnine.aes(x="Encoding_1", y="Encoding_2", fill="factor(Target)"),
        size=2,
        color="black",
    )
    + plotnine.xlab(xlab="Encoding dimension 1")
    + plotnine.ylab(ylab="Encoding dimension 2")
    + plotnine.ggtitle(title="MNIST Conv autoencoder with 2-dimensional encoding")
    + plotnine.theme_matplotlib()
)


In [ ]:
cluster_plot_mnist_conv

#### 4.3.2 Compare with plot of Linear Autoencoder

In [ ]:
cluster_plot_mnist_linear

# CIFAR 10 Conv Autoencoder

## 1. Download Weights of Pretrained model

In [ ]:
!gdown --id 1-WIgExr9O19tFgv_37a5tvGzwp4AixGa

## 2. Data Preparation

In [ ]:
# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# load the training and test datasets
train_data = datasets.CIFAR10(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.CIFAR10(root='data', train=False,
                                  download=True, transform=transform)

### 2.1 Train Test split

In [ ]:
# Create training and test dataloaders

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)

## 3. Model Building

In [ ]:
class Autoencoder(nn.Module):

    def __init__(self):
        super(Autoencoder,self).__init__()
        
        self.encoder = nn.Sequential(
            # conv layer (depth from 1 --> 16), 3x3 kernels
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            # conv layer (depth from 16 --> 4), 3x3 kernels
            nn.Conv2d(16, 4, 3, padding=1),
            nn.ReLU(True),
            # pooling layer to reduce x-y dims by two; kernel and stride of 2
            nn.MaxPool2d(2, 2))

        self.decoder = nn.Sequential(  
            ## a kernel of 2 and a stride of 2 will increase the spatial dims by 2           
            nn.ConvTranspose2d(4, 16, 2, stride=2),
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 3, 2, stride=2),
            nn.Sigmoid())

    def forward(self,x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

### 3.1 Check if GPU available

In [ ]:
torch.cuda.is_available()

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

### 3.2 Initialize Model

In [ ]:
# initialize the NN
model = Autoencoder()
model = model.to(device)
print(model)

### 3.3 Load weights

In [ ]:
# Specify a path
PATH = "cifar_conv.pt"

# Load
model = torch.load(PATH)
model.eval()

## 4. Model Evaluation

### 4.1 Visualize Reconstructed Images

In [ ]:
# obtain one batch of test images
dataiter = iter(test_loader)
images, labels = next(dataiter)

output = model(images.to(device))
# prep images for display
images = images.numpy()

# output is resized into a batch of images

# use detach when it's an output that requires_grad
output = output.detach().cpu().detach().numpy()

# plot the first ten input images and then reconstructed images
fig, axes = plt.subplots(nrows=2, ncols=10, sharex=True, sharey=True, figsize=(25,4))

# input images on top row, reconstructions on bottom
for images, row in zip([images, output], axes):
    for img, ax in zip(images, row):
        # print(npimg.shape)
        img = np.transpose(img, (1, 2, 0))
        ax.imshow(img)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

### 4.2 Visualize embedding space using t-SNE plots

In [ ]:
tsne = TSNE(n_components=2)
test_encoded_df = pd.DataFrame(columns = ['Encoding_1', 'Encoding_2', 'Target'])
outputs = []
labels = []

for data in test_loader:
        # _ stands in for labels, here
        images, label = data
        output = model(images.to(device))

        for i in range(0, batch_size):
            # output is resized into a batch of images
            output_n = output[i]
            lab = label[i].item()
            
            outputs.append(output_n.detach().cpu().detach().numpy().flatten())
            labels.append(lab)

tsne_op = tsne.fit_transform(outputs)

In [ ]:
for i in range(0, len(labels)):
  test_encoded_df = test_encoded_df.append({'Encoding_1' :tsne_op[i][0] , 'Encoding_2' :tsne_op[i][1], 'Target' : labels[i]},
ignore_index = True)

In [ ]:
cluster_plot_cifar_conv = (
    plotnine.ggplot(data=test_encoded_df)
    + plotnine.geom_point(
        mapping=plotnine.aes(x="Encoding_1", y="Encoding_2", fill="factor(Target)"),
        size=2,
        color="black",
    )
    + plotnine.xlab(xlab="Encoding dimension 1")
    + plotnine.ylab(ylab="Encoding dimension 2")
    + plotnine.ggtitle(title="CIFAR Conv autoencoder with 2-dimensional encoding")
    + plotnine.theme_matplotlib()
)


In [ ]:
cluster_plot_cifar_conv

### 4.2.1 Compare with t-SNE plot of Linear Auto Encoder

In [ ]:
cluster_plot_cifar_linear